# Script 1 - Clusters on *In-Situ* Saildrone Points

In [ ]:
#basic packages
import xarray as xr
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Rectangle
import matplotlib as mpl
import gsw
import seaborn as sns
import glob

#clustering packages
import itertools
from scipy import linalg, interpolate
from scipy.interpolate import griddata
from scipy.spatial import ConvexHull
from sklearn import mixture
from sklearn.neighbors import NearestCentroid
import statsmodels.api as sm

#map packages
from shapely.geometry import mapping
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature
import cartopy.crs as ccrs

#set random seed for reproducibility
SEED = 7

In [ ]:
#read in data
saildrone = xr.open_dataset('../data/saildrone_westcoast_combined.nc')
saildrone.close()

#resample per day
saildrone = saildrone.to_dataframe().groupby('relativeID').resample('1D').mean(numeric_only = True).drop('relativeID', axis = 1).reset_index('relativeID').to_xarray()
saildrone = saildrone.rename({'SAL_MEAN':'SAL_CTD_MEAN'})

#extract out the year/time
saildrone = saildrone.where(((saildrone.time >= pd.to_datetime('2018-07-01')) & (saildrone.time <= pd.to_datetime('2018-09-30'))) |
                            ((saildrone.time >= pd.to_datetime('2019-07-01')) & (saildrone.time <= pd.to_datetime('2019-09-30'))), drop = True)

#set lower bounds of salinity to be 30 PSU
saildrone = saildrone.where(saildrone.SAL_CTD_MEAN >= 30, drop = True)
saildrone

In [ ]:
###Add density calculations
#convert to dataframe, calculate density using gsw package, and convert back to xarray
saildrone_df = saildrone.to_dataframe()
saildrone_df['DENSITY_MEAN'] = gsw.sigma0(saildrone.SAL_CTD_MEAN.values, saildrone.TEMP_CTD_MEAN.values)
saildrone = saildrone_df.to_xarray()

#extract ou the minimum and maximum temp/salinity values
mint=np.min(saildrone_df['TEMP_CTD_MEAN'])
maxt=np.max(saildrone_df['TEMP_CTD_MEAN'])

mins=np.min(saildrone_df['SAL_CTD_MEAN'])
maxs=np.max(saildrone_df['SAL_CTD_MEAN'])

#create an evenly spaced series based on the range of values
#tempL=np.linspace(mint-1,maxt+1,156)
#salL=np.linspace(mins-0.25,maxs+0.25,156)
tempL = np.linspace(9,24)
salL = np.linspace(30.5,35)

#create a meshgride and fill linspace with density values
Tg, Sg = np.meshgrid(tempL,salL)
sigma_theta = gsw.sigma0(Sg, Tg)
cnt = np.linspace(sigma_theta.min(), sigma_theta.max(),156)

saildrone

### Time of Year Graphs

In [ ]:
###Temporal graph (day of year) by latitude for two years
plt.figure(figsize = (11, 4))
for i, color in zip([2018, 2019], ['firebrick', 'royalblue']):
    #saildrone_temp = saildrone.where(saildrone.relativeID != 0, drop = True)
    saildrone_temp = saildrone.where(saildrone.time.dt.year == i, drop = True)
    plt.scatter(saildrone_temp.time.dt.dayofyear, saildrone_temp.latitude, c = color, s = 20, label = i)

plt.title('Temporal Range of Saildrone Cruises with Latitude')
plt.ylabel('Latitude [°N]')
plt.legend(title = 'Year', loc='center left', bbox_to_anchor=(1, 0.5))
plt.grid(color = 'grey', linestyle = 'dotted', alpha = 0.7)
plt.xticks(range(182, 274, 7), pd.date_range(pd.to_datetime(182-1, unit='D', origin=str(2018)), pd.to_datetime(273-1, unit='D', origin=str(2018)), freq = '7D').strftime('%b-%d'))
plt.savefig('../figures/Figure1_saildrone_temporalrange_dayofyear.png', bbox_inches = 'tight', dpi = 300)   

In [ ]:
###Map of raw values colored by year
#define latitude and longitude boundaries
latr = [min(saildrone['latitude']), max(saildrone['latitude'])] 
lonr = [min(saildrone['longitude']), max(saildrone['longitude'])] 

# Select a region of our data, giving it a margin
margin = 0.75
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]-margin,lonr[1]+margin]]) 

#add state outlines
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='10m',
        facecolor='none')

# Create and set the figure context
# Create and set the figure context
fig = plt.figure(figsize=(10,6), dpi = 72) 
ax = plt.axes(projection=ccrs.PlateCarree()) 
ax.coastlines(resolution='10m',linewidth=1,color='black') 
ax.add_feature(cfeature.LAND, color='lightgrey', alpha=1)
ax.add_feature(states_provinces, linewidth = 0.5)
ax.add_feature(cfeature.BORDERS)
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 

gl = ax.gridlines(linestyle = 'dashed', linewidth = 0.5, alpha = 0.8, zorder = 5, draw_labels=True, y_inline = False, x_inline = False)
gl.top_labels = False
gl.right_labels = False

# ax.set_xticks(np.round([*np.arange(region_raw[1,0],region_raw[1,1]+1,1)][::-1],0), crs=ccrs.PlateCarree()) 
# ax.set_yticks(np.round([*np.arange(np.floor(region_raw[0,0]),region_raw[0,1]+1,1)],1), crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())

# Plot track data, color by temperature
for i, color in zip([2018, 2019], ['firebrick', 'royalblue']):
    #saildrone_temp = saildrone.where(saildrone.relativeID != 0, drop = True)
    saildrone_temp = saildrone.where(saildrone.time.dt.year == i, drop = True)
    plt.scatter(saildrone_temp.longitude, saildrone_temp.latitude, c = color, s = 12, label = i, alpha = 0.9, zorder = 6)

#plt.scatter(x = saildrone.longitude, y = saildrone.latitude, c = saildrone.time.dt.year, cmap = cmap, s = 10, alpha = 0.8)
legend = plt.legend(title = 'Year', loc='upper right')
plt.gca().set_aspect('equal')
plt.title('West Coast Saildrone Cruises by Year', fontdict = {'fontsize' : 14})
plt.savefig('../figures/Figure1_saildrone_map_by_year.png', bbox_inches = 'tight', dpi = 300)
plt.show()

### T-S Diagram

In [ ]:
###TS of all data
fig, ax = plt.subplots(figsize=(7.5,4))

#add density lines
positions = [(31.25, 22), (32.5, 23), (34, 23), (34.75, 20), (34.75, 17), (34.75, 12)]
cs = ax.contour(Sg, Tg, sigma_theta, colors='darkgrey', zorder=1, alpha = 0.8)
fig.draw_without_rendering()
cl=plt.clabel(cs,fontsize=10,inline=True,fmt='%.0f', manual = positions)

#create scatter plot colored by density
plt.scatter(y = saildrone.TEMP_CTD_MEAN, x = saildrone.SAL_CTD_MEAN, alpha = 0.8, s = 10, c = saildrone.DENSITY_MEAN)
plt.title('Temperature and Salinity \nfor West Coast Saildrone Cruises')
plt.ylabel('Temperature [°C]')
plt.xlabel('Salinity [PSU]')
clb=plt.colorbar(pad = 0.02)
clb.ax.set_ylabel(r'Density [kg m$^{-3}$]')
plt.ylim(9,24)
plt.xlim(30.5,35)
plt.xticks([*range(31, 36, 1)])
#save figure
# plt.savefig('../figures/saildrone_rawTS.png', bbox_inches = 'tight', dpi = 150)

## Clustering of TS Points

In [ ]:
###Extract out temp and salinity variables
#create array with temp and salinity data
X = saildrone.to_dataframe()[['TEMP_CTD_MEAN','SAL_CTD_MEAN']].dropna(axis = 0).to_numpy()
X[0:10]

In [ ]:
###Look at Bayesian Information Criterion (BIC) Scores
lowest_bic = np.infty
bic = []
n_components_range = range(1, 7)
cv_types = ["spherical", "tied", "diag", "full"]
for cv_type in cv_types:
    for n_components in n_components_range:
        # Fit a Gaussian mixture with EM
        gmm = mixture.GaussianMixture(
            n_components=n_components, covariance_type=cv_type, random_state = SEED
        )
        gmm.fit(X)
        bic.append(gmm.bic(X))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm

bic = np.array(bic)
color_iter = itertools.cycle(["navy", "turquoise", "cornflowerblue", "darkorange"])
clf = best_gmm
bars = []
Y_ = clf.predict(X)

# Plot the BIC scores
plt.figure(figsize=(8, 12))
spl = plt.subplot(2, 1, 1)
for i, (cv_type, color) in enumerate(zip(cv_types, color_iter)):
    xpos = np.array(n_components_range) + 0.2 * (i - 2)
    bars.append(
        plt.bar(
            xpos,
            bic[i * len(n_components_range) : (i + 1) * len(n_components_range)],
            width=0.2,
            color=color,
        )
    )
plt.xticks(n_components_range)
plt.ylim([bic.min() * 1.01 - 0.01 * bic.max(), bic.max()])
plt.title("BIC score per model")
xpos = (
    np.mod(bic.argmin(), len(n_components_range))
    + 0.65
    + 0.2 * np.floor(bic.argmin() / len(n_components_range))
)
plt.text(xpos, bic.min() * 0.97 + 0.03 * bic.max(), "*", fontsize=14)
spl.set_xlabel("Number of components")
spl.legend([b[0] for b in bars], cv_types)

In [ ]:
###TS plot with clusters
#create a figure
fig, ax = plt.subplots(figsize=(5,3))

#add density lines
positions = [(31.25, 22), (32.5, 23), (34, 23), (34.75, 20), (34.75, 17), (34.75, 12)]
cs = ax.contour(Sg, Tg, sigma_theta, colors='darkgrey', zorder=1, alpha = 0.8)
fig.draw_without_rendering()
cl=plt.clabel(cs,fontsize=10,inline=True,fmt='%.0f', manual = positions)

#add the clusters to the data
saildrone_df = saildrone.to_dataframe().dropna()
saildrone_df['GMM_Labels'] = Y_

#add colors to dataframe
colors = pd.DataFrame({'GMM_Labels': range(6), 'color': ["navy", "turquoise", "cornflowerblue", "darkorange", "purple", "slategrey"]})
saildrone_df = saildrone_df.merge(colors)

#plot points colored by cluster
plt.scatter(y = saildrone_df['TEMP_CTD_MEAN'], x = saildrone_df['SAL_CTD_MEAN'], c = saildrone_df['color'], s = 10, alpha = 0.8)

#add centroids
centroids = saildrone_df.groupby('GMM_Labels').mean(numeric_only = True)
plt.scatter(y = centroids['TEMP_CTD_MEAN'], x = centroids['SAL_CTD_MEAN'], c = 'yellow', s = 200, edgecolors='black', marker = '*', zorder = 2, label = 'Cluster Centroid') ###star centroid
# legend = plt.legend(loc = 'lower left')
# legend.get_frame().set_alpha(None)



plt.ylabel('Temperature [°C]')
plt.xlabel('Salinity [PSU]')
plt.subplots_adjust(hspace=0.35, bottom=0.02)
plt.ylim(9,24)
plt.xlim(30.5,35)
plt.xticks([*range(31, 36, 1)])
plt.title(r'$In$ $Situ$ Saildrone')
plt.savefig('../figures/Figure2C_saildrone_TS_with_GMMclusters_daily.png', bbox_inches = 'tight', dpi = 300)
plt.show()

#save output from centroids
centroids.to_csv('../data/saildrone_GMMcluster_centroids.csv')

In [ ]:
###Map with cluster colors
saildrone_df = saildrone.to_dataframe().dropna()
saildrone_df['GMM_Labels'] = Y_

#add colors to dataframe
colors = pd.DataFrame({'GMM_Labels': range(6), 'color': ["navy", "turquoise", "cornflowerblue", "darkorange", "purple", "slategrey"]})
saildrone_df = saildrone_df.merge(colors)

#define latitude and longitude boundaries
latr = [min(saildrone['latitude']), max(saildrone['latitude'])] 
lonr = [min(saildrone['longitude']), max(saildrone['longitude'])] 

# Select a region of our data, giving it a margin
margin = 0.75
region = np.array([[latr[0]-margin,latr[1]+margin],[lonr[0]-margin,lonr[1]+margin]]) 

#add state outlines
states_provinces = cfeature.NaturalEarthFeature(
        category='cultural',
        name='admin_1_states_provinces_lines',
        scale='50m',
        facecolor='none')

# Create and set the figure context
fig = plt.figure(figsize=(8,5), dpi = 72) 
ax = plt.axes(projection=ccrs.PlateCarree()) 
ax.coastlines(resolution='10m',linewidth=1,color='black') 
ax.add_feature(cfeature.LAND, color='grey', alpha=0.3)
ax.add_feature(states_provinces, linewidth = 0.5)
ax.add_feature(cfeature.BORDERS)
ax.set_extent([region[1,0],region[1,1],region[0,0],region[0,1]],crs=ccrs.PlateCarree()) 
gl = ax.gridlines(linestyle = 'dashed', linewidth = 0.5, alpha = 0.8, zorder = 10, draw_labels=True, y_inline = False, x_inline = False)
gl.top_labels = False
gl.right_labels = False

# ax.set_xticks(np.round([*np.arange(region_raw[1,0],region_raw[1,1]+1,1)][::-1],0), crs=ccrs.PlateCarree()) 
# ax.set_yticks(np.round([*np.arange(np.floor(region_raw[0,0]),region_raw[0,1]+1,1)],1), crs=ccrs.PlateCarree()) 
ax.xaxis.set_major_formatter(LongitudeFormatter(zero_direction_label=True))
ax.yaxis.set_major_formatter(LatitudeFormatter())


# Plot track data, color by temperature
cmap = (mpl.colors.ListedColormap(["navy", "turquoise", "cornflowerblue", "darkorange", "purple", "slategrey"])) #west coast
plt.scatter(x = saildrone_df.longitude, y = saildrone_df.latitude, c = saildrone_df.color, alpha = 0.9, s = 10)

plt.title(r'$In$ $Situ$ Saildrone', fontdict = {'fontsize' : 12}, x = 0.5)
plt.savefig('../figures/Figure2A_saildrone_map_with_GMMclusters.png', bbox_inches = 'tight', dpi = 300)
plt.show()

## Reclassification

In [ ]:
###Recalculate distances
#normalize data 
def NormalizeData(data, col):
    return (data[col] - np.min(saildrone_df[col])) / (np.max(saildrone_df[col]) - np.min(saildrone_df[col]))

saildrone_df['SAL_CTD_MEAN_NORM'] = NormalizeData(saildrone_df, 'SAL_CTD_MEAN')
saildrone_df['TEMP_CTD_MEAN_NORM'] = NormalizeData(saildrone_df, 'TEMP_CTD_MEAN')
centroids['SAL_CTD_MEAN_NORM'] = NormalizeData(centroids, 'SAL_CTD_MEAN')
centroids['TEMP_CTD_MEAN_NORM'] = NormalizeData(centroids, 'TEMP_CTD_MEAN')

dist0 = []
dist1 = []
dist2 = []
dist3 = []
dist4 = []
dist5 = []

#loop through each point in the saildrone data
for i in range(len(saildrone_df)):
    #calculate the distance to each centroid
    dist0.append(np.sqrt((((saildrone_df.iloc[i]['SAL_CTD_MEAN_NORM'] - centroids.iloc[0]['SAL_CTD_MEAN_NORM'])**2) + ((saildrone_df.iloc[i]['TEMP_CTD_MEAN_NORM'] - centroids.iloc[0]['TEMP_CTD_MEAN_NORM'])**2))))
    dist1.append(np.sqrt((((saildrone_df.iloc[i]['SAL_CTD_MEAN_NORM'] - centroids.iloc[1]['SAL_CTD_MEAN_NORM'])**2) + ((saildrone_df.iloc[i]['TEMP_CTD_MEAN_NORM'] - centroids.iloc[1]['TEMP_CTD_MEAN_NORM'])**2))))
    dist2.append(np.sqrt((((saildrone_df.iloc[i]['SAL_CTD_MEAN_NORM'] - centroids.iloc[2]['SAL_CTD_MEAN_NORM'])**2) + ((saildrone_df.iloc[i]['TEMP_CTD_MEAN_NORM'] - centroids.iloc[2]['TEMP_CTD_MEAN_NORM'])**2))))
    dist3.append(np.sqrt((((saildrone_df.iloc[i]['SAL_CTD_MEAN_NORM'] - centroids.iloc[3]['SAL_CTD_MEAN_NORM'])**2) + ((saildrone_df.iloc[i]['TEMP_CTD_MEAN_NORM'] - centroids.iloc[3]['TEMP_CTD_MEAN_NORM'])**2))))
    dist4.append(np.sqrt((((saildrone_df.iloc[i]['SAL_CTD_MEAN_NORM'] - centroids.iloc[4]['SAL_CTD_MEAN_NORM'])**2) + ((saildrone_df.iloc[i]['TEMP_CTD_MEAN_NORM'] - centroids.iloc[4]['TEMP_CTD_MEAN_NORM'])**2))))
    dist5.append(np.sqrt((((saildrone_df.iloc[i]['SAL_CTD_MEAN_NORM'] - centroids.iloc[5]['SAL_CTD_MEAN_NORM'])**2) + ((saildrone_df.iloc[i]['TEMP_CTD_MEAN_NORM'] - centroids.iloc[5]['TEMP_CTD_MEAN_NORM'])**2))))

#combine into a dataframe
distances = pd.DataFrame({'Cluster_0':dist0, 'Cluster_1':dist1,
                         'Cluster_2':dist2, 'Cluster_3':dist3,
                         'Cluster_4':dist4, 'Cluster_5':dist5})

#add in a relative id column
distances = distances.reset_index()

#convert from wide to long
distances = distances.melt(id_vars=['index'], value_vars=['Cluster_0', 'Cluster_1', 'Cluster_2', 'Cluster_3', 'Cluster_4', 'Cluster_5'], value_name = 'distance', var_name = 'GMM_Labels_new')

#select the smallest one from each point (ie the nearest centroid)
distances = distances.loc[distances.groupby('index').distance.idxmin()].set_index('index')#.reset_index(drop=True).set_index('index')

#join with the saildrone data
saildrone_reclass = saildrone_df.join(distances)
saildrone_reclass


In [ ]:
###TS plot with new clusters
#create a figure
fig, ax = plt.subplots(figsize=(5,3))

#add density lines
positions = [(31.25, 22), (32.5, 23), (34, 23), (34.75, 20), (34.75, 17), (34.75, 12)]
cs = ax.contour(Sg, Tg, sigma_theta, colors='darkgrey', zorder=1, alpha = 0.8)
fig.draw_without_rendering()
cl=plt.clabel(cs,fontsize=10,inline=True,fmt='%.0f', manual = positions)

#add colors to dataframe
colors = pd.DataFrame({'GMM_Labels_new': saildrone_reclass['GMM_Labels_new'].sort_values().unique(), 'color_new': ["navy", "turquoise", "cornflowerblue", "darkorange", "purple", "slategrey"]})
saildrone_reclass_col = saildrone_reclass.merge(colors, on = 'GMM_Labels_new')

#plot points colored by cluster
plt.scatter(y = saildrone_reclass_col['TEMP_CTD_MEAN'], x = saildrone_reclass_col['SAL_CTD_MEAN'], c = saildrone_reclass_col['color_new'], s = 10, alpha = 0.8)

#add centroids
centroids = saildrone_df.groupby('GMM_Labels').mean(numeric_only = True)
plt.scatter(y = centroids['TEMP_CTD_MEAN'], x = centroids['SAL_CTD_MEAN'], c = 'yellow', s = 300, edgecolors='black', marker = '*', zorder = 2) ###star centroid   
centroids = centroids.reset_index()
centroids['GMM_Labels_new'] = ['Cluster_' + str(i) for i in range(6)]

saildrone_reclass_col = saildrone_reclass_col.merge(centroids.reset_index()[['GMM_Labels_new', 'TEMP_CTD_MEAN', 'SAL_CTD_MEAN']].rename({'TEMP_CTD_MEAN':'TEMP_centroid', 'SAL_CTD_MEAN':'SAL_centroid'}, axis = 1))

#add lines
for ids, val in saildrone_reclass_col.iterrows():
    y = [val.TEMP_CTD_MEAN, val.TEMP_centroid]
    x = [val.SAL_CTD_MEAN, val.SAL_centroid]
    plt.plot(x, y, c = val.color_new, alpha = 0.15, zorder = 1)

#add title and format axes
plt.title(
    f"GMM Cluster Results: {gmm.covariance_type} model, "
    f"{gmm.n_components} components"
)
plt.ylabel('Temperature [°C]')
plt.xlabel('Salinity [PSU]')
plt.subplots_adjust(hspace=0.35, bottom=0.02)
plt.ylim(9,24)
plt.xlim(30.5,35)
plt.xticks([*range(31, 36, 1)])
plt.title(r'Reclassified $In$ $Situ$ Saildrone')
plt.savefig('../figures/Figure2D_saildrone_TS_with_GMMclusters_daily_reclassified.png', bbox_inches = 'tight')
plt.show()
